In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_DEPOT_PATH = /home/manabu/.julia-1.7.3


In [2]:
run(`conda --version`)

conda 4.8.3


Process(`conda --version`, ProcessExited(0))

In [3]:
run(`conda info --envs`)

# conda environments:
#
base                     /opt/anaconda3
jupyter               *  /opt/anaconda3/envs/jupyter



Process(`conda info --envs`, ProcessExited(0))

In [4]:
run(`conda list -n jupyter`)

# packages in environment at /opt/anaconda3/envs/jupyter:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
alsa-lib                  1.2.6.1              h7f98852_0    conda-forge
argon2-cffi               21.3.0             pyhd8ed1ab_0    conda-forge
argon2-cffi-bindings      21.2.0           py38h0a891b7_2    conda-forge
asttokens                 2.0.5              pyhd8ed1ab_0    conda-forge
attr                      2.5.1                h166bdaf_0    conda-forge
attrs                     21.4.0             pyhd8ed1ab_0    conda-forge
backcall                  0.2.0              pyh9f0ad1d_0    conda-forge
backports                 1.0                        py_2    conda-forge
backports.functools_lru_cache 1.6.4              pyhd8ed1ab_0    conda-forge
beautifulsoup4            4.11.1             pyha770c72_0    conda

Process(`conda list -n jupyter`, ProcessExited(0))

In [4]:
using PyCall

In [11]:
py_sys = pyimport("sys")
py_sys.version_info

(3, 8, 13, "final", 0)

# Why using...

## Pythonを使う理由

* `int` が多倍長なので桁溢れを気にせずプログラミングできる
* `numpy` が使える

In [56]:
py"type"(BigInt(2)^129)

PyObject <class 'int'>

In [53]:
string(typemax(UInt128), base=10)

"340282366920938463463374607431768211455"

In [55]:
BigInt(2)^129

680564733841876926926749214863536422912

## Juliaを使う理由

* `numpy` を使わなくても早い
* `numpy` も使える
* `Cxx.jl` が使えた・・・また使えるようになる

* [Integers and Floating-Point Numbers · The Julia Language](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/)

In [6]:
typemin(Int32), typemax(Int32)

(-2147483648, 2147483647)

In [7]:
typemin(Int64), typemax(Int64)

(-9223372036854775808, 9223372036854775807)

In [8]:
typemin(Int128), typemax(Int128)

(-170141183460469231731687303715884105728, 170141183460469231731687303715884105727)

* [Iteration utilities · The Julia Language](https://docs.julialang.org/en/v1/base/iterators/)

In [12]:
itr = (i for i in 1:100)

Base.Generator{UnitRange{Int64}, typeof(identity)}(identity, 1:100)

In [14]:
collect(Iterators.take(itr,10))

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

* "Dot Syntax for Vectorizing Functions"
  - [Functions · The Julia Language](https://docs.julialang.org/en/v1/manual/functions/#man-vectorized-1)

In [17]:
#import Pkg; Pkg.add("LLVM")
using LLVM

In [18]:
InitializeAllTargets()

In [27]:
name.(collect(targets()))

2-element Vector{String}:
 "x86-64"
 "x86"

In [24]:
targets()

LLVM.TargetIterator()

In [20]:
collect(targets())

2-element Vector{Target}:
 LLVM.Target(x86-64): 64-bit X86: EM64T and AMD64
 LLVM.Target(x86): 32-bit X86: Pentium-Pro and above

In [22]:
typeof(name)

typeof(name) (singleton type of function name, subtype of Function)

In [23]:
name

name (generic function with 10 methods)

In [25]:
methods(name)

# 10 methods for generic function "name":
[1] name(typ::DIType) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/debuginfo.jl:127
[2] name(scope::DIScope) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/debuginfo.jl:69
[3] name(structtyp::LLVM.StructType) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/type.jl:199
[4] name(mod::LLVM.Module) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/module.jl:47
[5] name(node::NamedMDNode) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/module.jl:88
[6] name(bb::BasicBlock) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/basicblock.jl:31
[7] name(val::Value) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/value.jl:50
[8] name(intr::Intrinsic) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/function.jl:181
[9] name(intr::Intrinsic, params::Vector{<:LLVMType}) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/core/function.jl:187
[10] name(t::Target) in LLVM at /home/manabu/.julia-1.7.3/packages/LLVM/WjSQG/src/target.jl:38

## SX-Auroraで使う理由

* オフローディングすると`numpy`やJuliaよりも圧倒的に (20倍くらい!?) 早い